In [0]:
from keras.applications.resnet50 import ResNet50,preprocess_input
from keras.preprocessing import image
from keras.layers import GlobalAveragePooling2D,Dense
from keras.models import Model
from keras.callbacks import EarlyStopping,ReduceLROnPlateau
from keras.backend import function
import numpy as np
import cv2
import matplotlib.pyplot as plt

#Declaring variables
n_classes=2
num_epochs=1
image_size=(500,368)
y=np.array([[1,0]])

#Inputting and Preprocessing of image
def preprocessing_input(img_input):
  img=image.load_img(img_input,target_size=image_size)
  x=image.img_to_array(img)
  x=x.reshape((1,x.shape[0],x.shape[1],x.shape[2]))
  x=preprocess_input(x)
  return x

#Defining model and adding GAP
def model(image):
  resnet_model=ResNet50(weights='imagenet',include_top=False,input_shape=(500,368,3))
  output=resnet_model.layers[-3].output
  output=GlobalAveragePooling2D()(output)
  output=Dense(n_classes,activation='sigmoid')(output)
  model=Model(inputs=resnet_model.inputs,outputs=output)
  model.summary()
  for layer in model.layers[:-10]:
    layer.trainable=False
  model.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])
  early_stop = EarlyStopping(monitor='loss', patience=5, verbose=1)
  reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2,patience=5, min_lr=0.0001)
  history = model.fit(image,y,epochs=num_epochs,verbose=1,callbacks=[early_stop,reduce_lr])
  model.save('Resnet.h5')
  return model

def create_activationmap(model,input_image):
  class_weight=model.layers[-1].get_weights()[0]
  final_conv_layer=model.layers[-3]
  output=function([model.layers[0].input],[final_conv_layer.output,model.layers[-1].output])
  [conv_outputs,prediction]=output([preprocessing_input(input_image)])
  conv_outputs=conv_outputs[0,:,:,:]
  cam = np.zeros(dtype=np.float32, shape=conv_outputs.shape[0:2])
  for index, weight in enumerate(class_weight[:, 1]):
    cam += weight * conv_outputs[:, :, index]
  cam /= np.max(cam)
  cam = cv2.resize(cam, image_size)
  heatmap = cv2.applyColorMap(np.uint8(255 * cam), cv2.COLORMAP_JET)
  heatmap[np.where(cam < 0.2)] = 0
  img=image.load_img(input_image,target_size=(368,500))
  img=image.img_to_array(img)
  img = heatmap * 0.5 + img
  cv2.imwrite("heatmap.jpg", img)
  heatmap = cv2.imread("heatmap.jpg")
  scaled_image = (((img - img.min()) * 255) / (img.max() - img.min())).astype(np.uint8)
  return plt.imshow(heatmap)


def main():
  preprocessed_img=preprocessing_input('snake.jpg')
  final_model=model(preprocessed_img)
  create_activationmap(final_model,'snake.jpg')

if __name__=='__main__':
  main()